In [ ]:
!unzip cancer.zip

In [3]:
from typing import List, Dict, Any
import pandas as pd
import numpy as np
import os

In [ ]:
train_df = pd.read_csv(os.path.join('data', "train.csv"))
train_df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [ ]:
from sklearn.preprocessing import LabelEncoder, StandardScaler

train_df = train_df.drop(columns=['PassengerId', 'Name', 'Ticket', 'Fare', 'Cabin']).dropna()
train_df["Sex"] = LabelEncoder().fit_transform(train_df["Sex"])
train_df["Embarked"] = LabelEncoder().fit_transform(train_df["Embarked"])
train_df['Age'] = StandardScaler().fit_transform(train_df['Age'].values.reshape(-1, 1))
train_df['bias'] = np.ones(train_df.shape[0])

In [ ]:
train_df

,Survived,Pclass,Sex,Age,SibSp,Parch,Embarked,bias
0,0,3,1,-0.527669,1,0,2,1.0
1,1,1,0,0.577094,1,0,0,1.0
2,1,3,0,-0.251478,0,0,2,1.0
3,1,1,0,0.369951,1,0,2,1.0
4,0,3,1,0.369951,0,0,2,1.0
...,...,...,...,...,...,...,...,...
885,0,3,0,0.646142,0,5,1,1.0
886,0,2,1,-0.182430,0,0,2,1.0
887,1,1,0,-0.734812,0,0,2,1.0
889,1,1,1,-0.251478,0,0,0,1.0


In [ ]:
from sklearn.model_selection import train_test_split

X = train_df.drop(columns=['Survived'])
y = train_df['Survived']

X_train, X_test, y_train, y_test = train_test_split(X.values, y.values, test_size=0.15, random_state=69)

In [ ]:
import torch
import numpy as np

In [ ]:
X = np.array([1])
Y = np.array([0])
Z = X + Y
X ,Y , Z

(array([1]), array([0]), array([1]))

# Mnist

In [ ]:
print('Penis')


Penis


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor

In [ ]:
transform = ToTensor()
mnist_ds = MNIST(root = 'data' ,download=True, transform=transform)
train_ds, val_ds = torch.utils.data.random_split(mnist_ds, [0.85, 0.15])

image, label = train_ds[0]


Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 9.91M/9.91M [00:02<00:00, 4.39MB/s]


Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 28.9k/28.9k [00:00<00:00, 63.1kB/s]


Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 1.65M/1.65M [00:01<00:00, 1.24MB/s]


Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 4.54k/4.54k [00:00<00:00, 8.81MB/s]

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



In [ ]:
image

tensor([[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,

In [ ]:
train_data_loader = DataLoader(train_ds, batch_size=256, shuffle=True)
val_data_loader = DataLoader(val_ds, batch_size=256, shuffle=False)

# Mnist

In [ ]:
#Model
class MnistModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.linear1 = torch.nn.Linear(28*28, 512)
        self.linear2 = torch.nn.Linear(512, 128)
        self.linear3 = torch.nn.Linear(128, 32)
        self.linear4 = torch.nn.Linear(32, 10)

    def forward(self, a):
        a = torch.relu(self.linear1(a))
        a = torch.relu(self.linear2(a))
        a = torch.relu(self.linear3(a))
        a = torch.sigmoid(self.linear4(a))
        return a

# Training Loop
# Loss, Optimizer, epochs, learning rate, seed

torch.manual_seed(69)
model = MnistModel().cuda()
learning_rate = 0.01
epochs = 10
loss = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


In [ ]:
# def accuracy_with_thresholding_mnist(y_true, y_pred, threshold = 0.5):
#     y_pred = (y_pred > 0.5).int() # Thresholding
#     anded = torch.logical_and(y_pred, y_val) # Element wise AND operation (checking both y_pred and y_true are the same value)
#     sum_accurate = torch.sum(anded).float() # Get the number of times y_pred and y_true agreed
#     sum_accurate /= len(y_pred) # Calculate accuracy for the batch and return it
#     return sum_accurate
def accuracy_no_one_hot(y_pred,y_true):
    _, predicted = y_pred.max(2)
    total = y_true.size(0)
    correct = predicted.eq(y_true).sum().item()
    return correct/total

In [ ]:
from torch.nn.functional import one_hot
max_accuracy = 0
for i in range(epochs):
    for idx, (x_batch, y_batch) in enumerate (train_data_loader):
        y_batch = y_batch.reshape(-1, 1)
        optimizer.zero_grad()
        y_pred = model(x_batch.reshape(-1, 1, 784).cuda())
        loss_value = loss(y_pred.cpu(), one_hot(y_batch, 10).float())
        loss_value.backward()
        optimizer.step()
        if idx % 20 == 0:
            val_loss = 0
            val_accuracy = 0
            for x_val, y_val in val_data_loader:
                y_pred = model(x_val.reshape(-1, 1, 784).cuda())
                y_val = y_val.reshape(-1, 1)
                val_loss += loss(y_pred.cpu(), one_hot(y_val, 10).float())
                val_accuracy += accuracy_no_one_hot(y_pred.cpu(), y_val.float())
            val_accuracy /= len(val_data_loader)
            val_loss /= len(val_data_loader)

            if val_accuracy > max_accuracy:
                max_accuracy = val_accuracy

            print(f"Epoch {i} - Training Loss: {loss_value.item()}, Validation Loss: {val_loss.item()}, Validation Accuracy: {val_accuracy}")

print(loss_value)
# Using the wrong accuract_with_thresholding function

In [ ]:
x_batch.shape

torch.Size([56, 1, 28, 28])

In [ ]:
one_hot(y_batch).shape

torch.Size([256, 10])

In [ ]:
y_batch.shape

torch.Size([256])

In [ ]:
y_val.shape

torch.Size([40, 1, 10])

In [ ]:
y_pred.shape

torch.Size([40, 1, 10])

In [ ]:
y_pred

tensor([[[0.0034, 0.0032, 0.0034,  ..., 0.0034, 0.0031, 0.0089]],

        [[0.0034, 0.0032, 0.0091,  ..., 0.0034, 0.0031, 0.0033]],

        [[0.0034, 0.0032, 0.0091,  ..., 0.0034, 0.0031, 0.0033]],

        ...,

        [[0.0034, 0.0032, 0.0034,  ..., 0.0034, 0.0031, 0.0033]],

        [[0.0034, 0.0032, 0.0034,  ..., 0.0093, 0.0031, 0.0033]],

        [[0.0034, 0.0032, 0.0034,  ..., 0.0034, 0.0086, 0.0089]]],
       device='cuda:0', grad_fn=<SoftmaxBackward0>)

In [ ]:
y_val

In [ ]:
y_pred = (y_pred > 0.5).int()

In [ ]:
y_pred

In [ ]:
anded = torch.logical_and(y_pred, y_val)
torch.sum(anded)

tensor(40)

In [ ]:
anded.shape

torch.Size([40, 1, 10])

In [ ]:
sum_accurate = torch.sum(anded).float()

In [ ]:
sum_accurate /= len(y_pred)

In [ ]:
sum_accurate

tensor(1.)

In [ ]:
for idx, (x_batch, y_batch) in enumerate (train_data_loader):
        y_batch = y_batch.reshape(-1, 1)
        optimizer.zero_grad()
        y_pred = model(x_batch.reshape(-1, 1, 784).cuda())
        loss_value = loss(y_pred.cpu(), one_hot(y_batch, 10).float())
        loss_value.backward()
        optimizer.step()
        if idx % 20 == 0:
            val_loss = 0
            val_accuracy = 0
            for x_val, y_val in val_data_loader:
                y_pred = model(x_val.reshape(-1, 1, 784).cuda())
                y_val = y_val.reshape(-1, 1)
                val_loss = loss(y_pred.cpu(), one_hot(y_val, 10).float())
                val_accuracy = accuracy_no_one_hot(y_pred.cpu(), y_val.float())
            val_accuracy /= len(val_data_loader)
            val_loss /= len(val_data_loader)

            if val_accuracy > max_accuracy:
                max_accuracy = val_accuracy

            print(f"Epoch {i} - Training Loss: {loss_value.item()}, Validation Loss: {val_loss.item()}, Validation Accuracy: {val_accuracy}")
            break

Epoch 1 - Training Loss: 0.010233274661004543, Validation Loss: 0.00016448496899101883, Validation Accuracy: 0.027083333333333334


In [ ]:
y_pred.shape

torch.Size([40, 1, 10])

In [ ]:
y_val.shape

torch.Size([40, 1])

In [ ]:
one_hot(y_val, 10).float().shape

torch.Size([40, 1, 10])

In [ ]:

_, predicted = y_pred.max(2)
total = y_val.size(0)
correct = predicted.cpu().eq(y_val).sum().item()

In [ ]:
predicted

In [ ]:
total

40

In [ ]:
correct

39

In [ ]:
y_val

tensor([[4],
        [9],
        [3],
        [0],
        [8],
        [4],
        [9],
        [7],
        [9],
        [3],
        [7],
        [7],
        [6],
        [7],
        [1],
        [1],
        [0],
        [4],
        [9],
        [6],
        [7],
        [0],
        [3],
        [7],
        [7],
        [5],
        [1],
        [5],
        [3],
        [5],
        [1],
        [4],
        [7],
        [0],
        [8],
        [4],
        [4],
        [3],
        [2],
        [2]])

In [ ]:
y_pred.max(2)[1]

tensor([[4],
        [9],
        [3],
        [0],
        [8],
        [4],
        [9],
        [7],
        [9],
        [3],
        [7],
        [7],
        [6],
        [7],
        [1],
        [1],
        [0],
        [4],
        [9],
        [6],
        [7],
        [0],
        [3],
        [7],
        [7],
        [5],
        [1],
        [5],
        [3],
        [5],
        [1],
        [4],
        [7],
        [0],
        [8],
        [4],
        [9],
        [3],
        [2],
        [2]], device='cuda:0')

In [ ]:
images, labels = next(iter(train_data_loader))
print("images-size:", images.shape)

print("out-size:", labels.shape)


images-size: torch.Size([256, 1, 28, 28])
out-size: torch.Size([256])


In [ ]:
x_batch[0]

tensor([[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0745, 0.7098, 0.9059, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.2000, 0.6314, 0.9255, 0.9882, 0.8196, 0.0000,

In [ ]:
x_batch[1]

tensor([[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,

tensor([[[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000,  0.0000,  0.0000,  0.0745,  0.7098,  0.9059,
           0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000,  0.0000,  0.0000,  0

# Mnist using CNN

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.functional import one_hot

In [ ]:
class CNNMnistModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 5, 3, stride=1, padding=1) # n, 5, 28, 28 where n = batch_size
        self.fc1 = nn.Linear(5 * 28 * 28, 512) # (3920, 512)
        self.fc2 = nn.Linear(512, 128)
        self.fc3 = nn.Linear(128, 10)
        # self.pool = nn.MaxPool2d(2, 2)
        # self.conv2 = nn.Conv2d(6, 16, 5)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = x.reshape(-1, 28*28*5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.sigmoid(self.fc3(x))
        return x
        # x = self.pool(F.relu(self.conv1(x)))
        # x = self.pool(F.relu(self.conv2(x)))
        # x = torch.flatten(x, 1) # flatten all dimensions except batch
        # x = F.relu(self.fc1(x))
        # x = F.relu(self.fc2(x))
        # x = self.fc3(x)
        # return x

In [ ]:
torch.manual_seed(69)
model = CNNMnistModel().cuda()
learning_rate = 0.001
epochs = 10
loss = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
def accuracy_no_one_hot(y_pred,y_true):
    _, predicted = y_pred.max(2)
    total = y_true.size(0)
    correct = predicted.eq(y_true).sum().item()
    return correct/total

In [ ]:
max_accuracy = 0
for i in range(epochs):
  for idx, (x_batch, y_batch) in enumerate(train_data_loader):
    # Training loop
    y_batch = y_batch.reshape(-1, 1)
    optimizer.zero_grad()
    y_pred = model(x_batch.reshape(-1, 1, 28, 28).cuda())
    loss_value = loss(y_pred.cpu().reshape(-1, 1, 10), one_hot(y_batch, 10).float())
    loss_value.backward()
    optimizer.step()
    # Validation loop
    if idx % 20 == 0:
      val_loss = 0
      val_accuracy = 0
      for x_val, y_val in val_data_loader:
        y_pred = model(x_val.reshape(-1, 1, 28, 28).cuda())
        y_val = y_val.reshape(-1, 1)
        val_loss += loss(y_pred.cpu().reshape(-1, 1, 10), one_hot(y_val, 10).float())
        val_accuracy += accuracy_no_one_hot(y_pred.cpu().reshape(-1, 1, 10), y_val.float())
      val_accuracy /= len(val_data_loader)
      val_loss /= len(val_data_loader)

      if val_accuracy > max_accuracy:
        max_accuracy = val_accuracy

      print(f"Epoch {i} - Training Loss: {loss_value.item()}, Validation Loss: {val_loss.item()}, Validation Accuracy: {val_accuracy}")

print(loss_value)

Epoch 0 - Training Loss: 0.6930078268051147, Validation Loss: 0.62453293800354, Validation Accuracy: 0.1212673611111111
Epoch 0 - Training Loss: 0.32423144578933716, Validation Loss: 0.31902551651000977, Validation Accuracy: 0.10219184027777778
Epoch 0 - Training Loss: 0.2889266610145569, Validation Loss: 0.28761398792266846, Validation Accuracy: 0.5281032986111112
Epoch 0 - Training Loss: 0.2076440155506134, Validation Loss: 0.20477065443992615, Validation Accuracy: 0.7328993055555555
Epoch 0 - Training Loss: 0.143526092171669, Validation Loss: 0.1279795914888382, Validation Accuracy: 0.8162760416666667
Epoch 0 - Training Loss: 0.11486707627773285, Validation Loss: 0.09807372838258743, Validation Accuracy: 0.8538628472222222
Epoch 0 - Training Loss: 0.07332531362771988, Validation Loss: 0.08025065809488297, Validation Accuracy: 0.8781032986111111
Epoch 0 - Training Loss: 0.08232925832271576, Validation Loss: 0.06836148351430893, Validation Accuracy: 0.9002387152777778
Epoch 0 - Traini

In [ ]:
y_pred.shape

torch.Size([256, 10])

# Carcenoma classification model using CNN

In [13]:
import glob
import pandas as pd

In [11]:
colon_aca = glob.glob("./lung_colon_image_set/colon_image_sets/*/*")

In [ ]:
colon_aca

In [18]:
df = pd.DataFrame(colon_aca, columns = ["path"])

In [19]:
df

,path
0,./lung_colon_image_set/colon_image_sets/colon_...
1,./lung_colon_image_set/colon_image_sets/colon_...
2,./lung_colon_image_set/colon_image_sets/colon_...
3,./lung_colon_image_set/colon_image_sets/colon_...
4,./lung_colon_image_set/colon_image_sets/colon_...
...,...
9995,./lung_colon_image_set/colon_image_sets/colon_...
9996,./lung_colon_image_set/colon_image_sets/colon_...
9997,./lung_colon_image_set/colon_image_sets/colon_...
9998,./lung_colon_image_set/colon_image_sets/colon_...


In [24]:
df2 = df["path"].str.split("/", expand = True)

In [27]:
labels = df2[3]

In [28]:
labels

,3
0,colon_aca
1,colon_aca
2,colon_aca
3,colon_aca
4,colon_aca
...,...
9995,colon_n
9996,colon_n
9997,colon_n
9998,colon_n


In [29]:
labeled_colon_data = df

In [30]:
labeled_colon_data["labels"] = labels

In [31]:
labeled_colon_data

,path,labels
0,./lung_colon_image_set/colon_image_sets/colon_...,colon_aca
1,./lung_colon_image_set/colon_image_sets/colon_...,colon_aca
2,./lung_colon_image_set/colon_image_sets/colon_...,colon_aca
3,./lung_colon_image_set/colon_image_sets/colon_...,colon_aca
4,./lung_colon_image_set/colon_image_sets/colon_...,colon_aca
...,...,...
9995,./lung_colon_image_set/colon_image_sets/colon_...,colon_n
9996,./lung_colon_image_set/colon_image_sets/colon_...,colon_n
9997,./lung_colon_image_set/colon_image_sets/colon_...,colon_n
9998,./lung_colon_image_set/colon_image_sets/colon_...,colon_n


In [ ]:
class DataSetClass(Dataset):
    def __init__(self):
      ...